# TODO
 Google PaLM API for langchain doesnt like anything other than the first message to be a system message, so i restructured the chat templates. The error message was pretty misleading.  
 Now the model doesn't follow the format instructions at all. It doesnt even attempt to conform to JSON format, much less the specific structure.

In [1]:
import os
import json
import time
import pprint as pp
import pandas as pd
from pydantic import BaseModel, Field
from typing import List, Dict
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from IPython.display import display, Markdown, JSON

In [2]:
load_dotenv(override=True) # loading API key from environment

True

## Read in data with annual curriculum plan

In [3]:
GSHEET_ID = os.getenv('GSHEET_ID')
google_sheet_url = f"https://docs.google.com/spreadsheets/d/{GSHEET_ID}/export?format=csv&gid=0"
annual_plan_df = pd.read_csv(google_sheet_url)
annual_plan_df.rename(columns=lambda x: x.replace(" ", "_").lower(), inplace=True)
annual_plan_df

,month,theme_to_explore,language_arts,math,social_studies,science
0,August,Getting to Know One Another; Class Community,"Personal narratives, reading comprehension",Counting and sorting,Our classroom community; citizenship and gover...,Exploring the school environment and local eco...
1,September,"On the Map, where we’re from",Descriptive writing (My Place); vocabulary dev...,latitutde and longitude; place value,My hometown; local history; geography,"Water cycle, rivers; Geology, caves, aquifers"
2,October,Fall; Halloween; Dia de los Muertos,Creative writing; grammar and punctuation,"units of measurement (length, weight, etc.); E...",Fall celebrations around the world; elections,Food chains and food webs; Local flora and fauna
3,November,Gratitude; Native Americans,Listening and speaking skills; Thankful Though...,Addition and subtraction,Native American history; Settlement of US and ...,Seasonal Changes and Plants; Winter Animal Ada...
4,December,Winter Solstice and winter holidays,book exchange; holiday cards; holiday play/skit,Geometry (2-D shapes); symmetry (snowflake),Winter holidays worldwide,"Physical science (matter, energy, force, motion)"
5,January,"Calendar, New Year, Goals, MLK",Dreams and goals; journaling; 'I Have a Dream'...,"Time units, Calendar math, multiplication",New Year's Celebrations worldwide; Community h...,"Climate zones, seasons, and weather"
6,February,"Black History, Love/Valentine's Day",African American biographies; exploring emotio...,"Fractions, Division; Number Patterns",African American Achievements; Civil Rights Mo...,"The human body, heart health and exercise"
7,March,Spring; Easter; Ramadan,Springtime poetry; nature observation,Patterns and Geometry; Geometry in Islamic Art,"Cultural Spring celebrations -- Easter, Ramadan",Life cycles of plants and animals (frogs and b...
8,April,Space; Passover,"Space stories, science fiction; constellation ...",Patterns and algebraic thinking; math in the s...,Passover origins; History of astronomy,Solar System and celestial bodies
9,May,Ocean Life,Ocean vocabulary; sea creatures; sea stories,Problem-solving and mathematical reasoning; oc...,Coastal ecosystems; exploration and navigation,Ocean zones; Marine life and habitats


# Create Prompt Templates

In [239]:
class ActivityIdea(BaseModel):
    name: str
    description: str

class ActivityIdeasOutput(BaseModel):
    ideas: List[ActivityIdea]
# Set up a parser + inject instructions into the prompt template.
activity_ideas_parser = PydanticOutputParser(pydantic_object=ActivityIdeasOutput)

In [257]:
idea_system_template = SystemMessagePromptTemplate.from_template(
    template= """
    You are a curriculum development expert and I am a teacher of second graders at a micro-school, with seven students of varying abilities.
    - The microschool stresses the importance of outdoor education and student-led development, but it also wants to ensure high academic standards.
    - Try to work in phonics work and literacy development into the language art activities.
    - Try to work in arithmetic and numeracy development into the math activities.
    - Keep in mind the grade level and number of the students when generating your response.
    
    I want you to help me brainstorm activity ideas for my classroom.
    Each activity could also be thought of as a lesson where the kids are learning valuable skills.
    I only need you to give me the activity names and a brief description of each idea.
    
    Here are some additional guidelines for generating these ideas:
    - Be creative in your activity ideas. Avoid common activities that have no direct relation to the lesson content, like scavenger hunts and word searches.
    - Each activity idea should be able to be accomplished in 30 mins-3 hours.
    - Each activity should relate directly to skills and concepts from the relevant academic subject.
    - The description should only be sentence or two and should not simply repeat the theme or the topic. It should describe the nature of the activity.

    {format_instructions}
    """
)

In [258]:
idea_human_template = HumanMessagePromptTemplate.from_template(
    template="""
    I want you to generate {num_ideas} activity ideas for the month of {month}. The theme for this month is {theme}.
    The academic subject is {subject} and the topic suggestions for this month are: {topic}.    
    """
)

In [259]:
idea_chat_prompt_template = ChatPromptTemplate(
    messages=[idea_system_template, idea_human_template],
    input_variables=["num_ideas", "month", "theme", "subject", "topic"],
    partial_variables={"format_instructions": activity_ideas_parser.get_format_instructions()}
)

In [260]:
print(activity_ideas_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ideas": {"title": "Ideas", "type": "array", "items": {"$ref": "#/definitions/ActivityIdea"}}}, "required": ["ideas"], "definitions": {"ActivityIdea": {"title": "ActivityIdea", "type": "object", "properties": {"name": {"title": "Name", "type": "string"}, "description": {"title": "Description", "type": "string"}}, "required": ["name", "description"]}}}
```


In [261]:
class LessonPlanOutput(BaseModel):
    time_estimate: str
    list_of_materials: List[str]
    procedure: List[str]
    additional_notes: List[str]
    differentiation_strategies: List[str]

# Set up a parser + inject instructions into the prompt template.
lesson_plan_parser = PydanticOutputParser(pydantic_object=LessonPlanOutput)

In [262]:
lesson_system_template = SystemMessagePromptTemplate.from_template(
    template= """
    You are a curriculum development expert and I am a teacher of second graders at a micro-school, with seven students of varying abilities.
    - The microschool stresses the importance of outdoor education and student-led development, but it also wants to ensure high academic standards.
    - Try to work in phonics work and literacy development into the language art activities.
    - Try to work in arithmetic and numeracy development into the math activities.
    - Keep in mind the grade level and number of the students when generating your response.
    
    I want you to generate a detailed lesson plan. I will give you a name and a short description of the lesson, as well as the academic subject the lesson plan should relate to.
    You will provide:
     - a time estimate
     - a list of materials
     - a step-by-step procedure for executing the lesson.
     - additional notes
     - differentiation strategies for evaluating students and providing additional challenge or assistance, and modifications for neurodivergent students
     
     Additional guidelines for the lesson plan:
      - Ensure that the lesson uses and develops skills relevant to the academic subject.
      - Try to make the best estimate of the total amount of time the activity will take. Be generous because these are young kids and they take a while!
      - The 'additional notes' specifies the actual content mentioned in the 'procedure' (e.g., interview questions, puzzle clues, specific writing prompts, etc.), so lesson plan provides everything needed to execute the lesson.
    
    {format_instructions}
    """
    )

In [267]:
lesson_human_template = HumanMessagePromptTemplate.from_template(
    template="""
    I want you to generate a *detailed* lesson plan for the academic subject of {subject}.
    The name of the lesson is {activity_name}, and the short description is: {activity_description}.
    """
)

In [264]:
    # - The 'differentiation strategies' provides modifications for students who need more support, for students who need more challenge, and for neurodivergent students.

In [265]:
    # - The 'additional notes' must **NOT** include differentiation strageties. There should be no overlap. Double-check this before giving me your response.

In [268]:
lesson_plan_chat_prompt_template = ChatPromptTemplate(
    messages=[lesson_system_template, lesson_human_template],
    input_variables=["activity_name", "activity_description", "subject"],
    partial_variables={"format_instructions": lesson_plan_parser.get_format_instructions()}
)

In [269]:
print(lesson_plan_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"time_estimate": {"title": "Time Estimate", "type": "string"}, "list_of_materials": {"title": "List Of Materials", "type": "array", "items": {"type": "string"}}, "procedure": {"title": "Procedure", "type": "array", "items": {"type": "string"}}, "additional_notes": {"title": "Additional Notes", "type": "array", "items": {"type": "string"}}, "differentiation_strategies": {"title": "Differentiation Strategies", "type": "array", "items": {"type": "string"}}}, "required": ["time_estimate", "list_of_materials", "procedure", "addition

# Test on Single Record

In [270]:
num_plans = 5

In [271]:
month = "September"
subject = 'math'

In [272]:
month_record = annual_plan_df.query("month == @month").to_dict(orient='records')[0]

In [273]:
theme = month_record['theme_to_explore']
topic = month_record[subject]

In [277]:
idea_prompt = idea_chat_prompt_template.format_prompt(
    num_ideas=num_plans,
    month=month,
    theme=theme,
    subject=subject, 
    topic=topic 
)

In [278]:
type(idea_prompt)

langchain.prompts.chat.ChatPromptValue

In [279]:
print(idea_prompt.messages[0].content)


    You are a curriculum development expert and I am a teacher of second graders at a micro-school, with seven students of varying abilities.
    - The microschool stresses the importance of outdoor education and student-led development, but it also wants to ensure high academic standards.
    - Try to work in phonics work and literacy development into the language art activities.
    - Try to work in arithmetic and numeracy development into the math activities.
    - Keep in mind the grade level and number of the students when generating your response.
    
    I want you to help me brainstorm activity ideas for my classroom.
    Each activity could also be thought of as a lesson where the kids are learning valuable skills.
    I only need you to give me the activity names and a brief description of each idea.
    
    Here are some additional guidelines for generating these ideas:
    - Be creative in your activity ideas. Avoid common activities that have no direct relation to the l

In [280]:
print(idea_prompt.messages[1].content)


    I want you to generate 5 activity ideas for the month of September. The theme for this month is On the Map, where we’re from.
    The academic subject is math and the topic suggestions for this month are: latitutde and longitude; place value.    
    


In [282]:
idea_messages = idea_prompt.to_messages()

In [283]:
type(idea_messages)

list

In [284]:
len(idea_messages)

2

In [285]:
type(idea_messages[0])

langchain.schema.messages.SystemMessage

In [286]:
type(idea_messages[1])

langchain.schema.messages.HumanMessage

In [287]:
# dummy = SystemMessage(content='you are a helpful assistant')

# idea_messages.insert(0, dummy)
# from langchain.prompts.chat import ChatPromptValue

# from langchain.schema.messages import SystemMessage

### Instantiate Test Chat Model

In [288]:
model_name = "gpt-3.5-turbo"
# chat_model = ChatOpenAI(model_name=model_name, temperature=0.1, max_tokens=3000)
# model_name = "text-davinci-003"
# model = OpenAI(model_name=model_name, temperature=temperature, max_tokens=3000)
from langchain.chat_models import ChatGooglePalm
chat_model = ChatGooglePalm(model_name='models/chat-bison-001', temperature=0.1, max_tokens=1000)

In [289]:
type(idea_messages[0])

langchain.schema.messages.SystemMessage

In [290]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [291]:
idea_response = chat_model(idea_messages)

In [293]:
idea_response.content

'Here are 5 activity ideas for the month of September, with the theme "On the Map, where we\'re from":\n\n1. **Latitude and Longitude**\n\n* **Activity:** Have students create a map of their neighborhood or community. They can use markers, crayons, or paint to color in the different areas. Then, they can use a compass to find the latitude and longitude of their home and other important places in their neighborhood.\n* **Learning Objectives:** Students will learn about latitude and longitude and how to use them to find locations on a map. They will also practice their map skills, such as using a compass and identifying different landmarks.\n* **Materials:** Paper, markers, crayons, paint, compass\n\n2. **Place Value**\n\n* **Activity:** Have students create a map of the world using place value blocks. They can use the blocks to represent the population of each country. Then, they can compare the populations of different countries and discuss why some countries have more people than othe

In [294]:
ideas_parsed = activity_ideas_parser.parse(response.content)

OutputParserException: Failed to parse ActivityIdeasOutput from completion Here are 5 activity ideas for the month of September, with a theme of "On the Map, where we're from":

* **Activity 1: Create a map of your neighborhood.**

Students will use a variety of materials, such as paper, markers, and string, to create a map of their neighborhood. They can label the important landmarks, such as their house, school, and park. This activity will help students learn about the geography of their community and develop their spatial reasoning skills.

* **Activity 2: Learn about latitude and longitude.**

Students will learn about latitude and longitude and how they are used to locate places on a map. They can use a globe or a map to practice finding different locations. This activity will help students develop their understanding of spatial relationships.

* **Activity 3: Play a game of "Where in the World?"**

Students will work in teams to guess the location of different places around the 

In [27]:
pp.pprint(ideas_parsed.dict())

{'ideas': [{'description': 'Students will create a visual representation of '
                           'their journey to school using latitude and '
                           'longitude coordinates.',
            'name': 'Mapping Our Journey'},
           {'description': 'Students will use a map to identify and locate '
                           'different landmarks in their neighborhood, '
                           'practicing their understanding of latitude and '
                           'longitude.',
            'name': 'Exploring Our Neighborhood'},
           {'description': 'Students will go on a scavenger hunt around the '
                           'school grounds, using latitude and longitude '
                           'coordinates to find hidden treasures.',
            'name': 'Latitude and Longitude Scavenger Hunt'},
           {'description': 'Students will create a cityscape using place value '
                           'blocks, representing buildings with diffe

In [28]:
idea = ideas_parsed.ideas[0]

In [29]:
idea_name = idea.name
idea_description = idea.description

In [30]:
print(f"{idea_name}: {idea_description}")

Mapping Our Journey: Students will create a visual representation of their journey to school using latitude and longitude coordinates.


In [31]:
lesson_plan_prompt_value = lesson_plan_chat_prompt_template.format_prompt(
    activity_name=idea_name,
    activity_description=idea_description,
    subject=subject
)

In [32]:
lp_messages = lesson_plan_prompt_value.to_messages()

In [33]:
lp_messages

[SystemMessage(content='\n    You are a curriculum development expert and I am a teacher of second graders at a micro-school, with seven students of varying abilities.\n    - The microschool stresses the importance of outdoor education and student-led development, but it also wants to ensure high academic standards.\n    - Try to work in phonics work and literacy development into the language art activities.\n    - Try to work in arithmetic and numeracy development into the math activities.\n    - Keep in mind the grade level and number of the students when generating your response.\n    ', additional_kwargs={}),
 SystemMessage(content='\n    I want you to generate a detailed lesson plan. I will give you a name and a short description of the lesson, as well as the academic subject the lesson plan should relate to.\n    You will provide:\n     - a time estimate\n     - a list of materials\n     - a step-by-step procedure for executing the lesson.\n     - additional notes\n     - differe

In [34]:
lp_response = chat_model(lp_messages)

In [35]:
lp_parsed = lesson_plan_parser.parse(lp_response.content)

In [36]:
lp_dict = lp_parsed.dict()

In [37]:
full_lp = {k: v for k, v in idea.dict().items()}
full_lp.update(lp_dict)

In [38]:
full_lp['additional_notes']

["Encourage creativity and artistic expression in the students' maps.",
 'Remind the students to use rulers to draw straight lines between the locations.',
 'Provide support and guidance to students who may struggle with reading and interpreting the latitude and longitude coordinates.',
 'For neurodivergent students, consider providing visual aids or manipulatives to help them understand the concept of latitude and longitude.']

In [39]:
full_lp['differentiation_strategies']

['For advanced students, challenge them to include additional significant locations on their maps and write more detailed descriptions.',
 'For students who need extra support, provide pre-marked latitude and longitude coordinates for them to use when mapping their journeys.',
 'For neurodivergent students, offer alternative ways for them to represent their journey, such as using symbols or pictures instead of latitude and longitude coordinates.']

## Define Markdown Helper Functions

In [40]:
def make_ordered_list_markdown(lst):
    markdown_list = [f"{idx}. {val}" for idx, val in enumerate(lst, start=1)]
    return '\n'.join(markdown_list)

In [41]:
def make_list_markdown(lst):
    return "- " + "\n- ".join(lst)

In [42]:
def convert_lesson_plan_to_markdown(lesson_plan_dict):
    res = []
    res.append(f"## {lesson_plan_dict['name']}")
    res.append(lesson_plan_dict['description'])
    res.append(f"\n### Time estimate: {lesson_plan_dict['time_estimate']}")
    res.append(f"\n### Materials")
    res.append(make_list_markdown(lesson_plan_dict["list_of_materials"]))
    res.append(f"\n### Procedure")
    res.append(make_ordered_list_markdown(lesson_plan_dict['procedure']))
    res.append(f"\n### Additional notes")
    res.append(make_list_markdown(lesson_plan_dict['additional_notes']))
    res.append(f"\n### Differentiation strategies")
    res.append(make_list_markdown(lesson_plan_dict['differentiation_strategies']))
    
    return "\n".join(res)
    

In [43]:
md = convert_lesson_plan_to_markdown(full_lp)

In [44]:
display(Markdown(md))

## Mapping Our Journey
Students will create a visual representation of their journey to school using latitude and longitude coordinates.

### Time estimate: 60 minutes

### Materials
- Large world map
- Markers or colored pencils
- Rulers
- Latitude and longitude coordinates handout
- Index cards or sticky notes

### Procedure
1. Introduce the concept of latitude and longitude to the students. Explain that latitude and longitude are coordinates used to locate places on a map.
2. Show the students a large world map and point out the lines of latitude (horizontal) and longitude (vertical). Explain that latitude measures distance north or south of the equator, while longitude measures distance east or west of the prime meridian.
3. Distribute the latitude and longitude coordinates handout to each student. Review the handout together, explaining how to read and interpret latitude and longitude coordinates.
4. Have the students think about their journey to school. Ask them to identify at least three significant locations along their route.
5. Instruct the students to use the latitude and longitude coordinates to locate each of the significant locations on the world map. They can use markers or colored pencils to mark the locations.
6. Once the students have marked their significant locations, have them use rulers to draw lines connecting the locations, creating a visual representation of their journey to school.
7. After the students have finished mapping their journeys, have them write a short description of each significant location on index cards or sticky notes. They can attach the cards or notes to the corresponding locations on the map.
8. Give the students an opportunity to share their maps and descriptions with the class. Encourage them to explain why each location is significant to their journey to school.
9. Wrap up the lesson by discussing the importance of maps and coordinates in navigation and understanding the world around us.

### Additional notes
- Encourage creativity and artistic expression in the students' maps.
- Remind the students to use rulers to draw straight lines between the locations.
- Provide support and guidance to students who may struggle with reading and interpreting the latitude and longitude coordinates.
- For neurodivergent students, consider providing visual aids or manipulatives to help them understand the concept of latitude and longitude.

### Differentiation strategies
- For advanced students, challenge them to include additional significant locations on their maps and write more detailed descriptions.
- For students who need extra support, provide pre-marked latitude and longitude coordinates for them to use when mapping their journeys.
- For neurodivergent students, offer alternative ways for them to represent their journey, such as using symbols or pictures instead of latitude and longitude coordinates.

# Define Wrapper Functions

In [45]:
def prompt_wrapper(prompt_template, parser, model, max_retries=5, **kwargs):
    prompt_value = prompt_template.format_prompt(**kwargs)
    messages = prompt_value.to_messages()
    for attempt in range(max_retries + 1):  #sometimes the parser fails inexplicably
        try:
            output = model(messages)
            parsed_output = parser.parse(output.content)
            break
        except Exception as e:
            if attempt < max_retries:
                print(e)
                print(f"Attempt {attempt + 1} failed. Retrying...")
                time.sleep(2)  # Wait before retrying
            else:
                print(f"API call failed after {max_retries} attempts. Skipping.")
    return parsed_output

In [46]:
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=temperature, max_tokens=3000)

In [47]:
# Generate activity ideas for each month-subject combo
activity_ideas_parsed = prompt_wrapper(  
                prompt_template=idea_chat_prompt_template, 
                parser=activity_ideas_parser,
                model=chat_model,
                num_ideas=num_plans,
                month=month,
                theme=theme,
                subject=subject, 
                topic=topic
            )

# examine output
pp.pprint(activity_ideas_parsed.dict())

{'ideas': [{'description': 'Students will create a visual representation of '
                           'their journey to school using latitude and '
                           'longitude coordinates.',
            'name': 'Mapping Our Journey'},
           {'description': 'Students will use latitude and longitude '
                           'coordinates to find hidden treasures around the '
                           'school campus.',
            'name': 'Latitude and Longitude Scavenger Hunt'},
           {'description': 'Students will create a city using place value '
                           'blocks to represent the different buildings and '
                           'landmarks.',
            'name': 'Place Value City'},
           {'description': 'Students will solve math puzzles related to maps '
                           'and geography, incorporating place value concepts.',
            'name': 'Map Math Puzzles'},
           {'description': 'Students will play a game of bi

In [48]:
def wrapper(annual_plan_df, activity_ideas_model, lesson_plan_model, num_plans=5, months=None):
    res = {}
    if months is None:
        months = annual_plan_df['month'].values
    df = annual_plan_df.query('month.isin(@months)')
    display(df)
    for idx, row in df.head(3).iterrows(): # first iterate through the months in the table
        month = row["month"]
        theme = row["theme_to_explore"]
        print(f"# {month}")
        markdown_lst = []
        markdown_lst.append(f"# {month}")
        markdown_lst.append(f"## Theme: {theme}")
        res[month] = dict()
        subjects = df.columns[2:].values
        for subject in subjects:  # for each month, iterate through the academic subjects
            pretty_subject = subject.replace("_", " ").title()
            markdown_lst.append(f"\n# {pretty_subject}")
            print(f"## {pretty_subject}")
            topic = row[subject]
            activity_ideas_parsed = prompt_wrapper(  # Generate activity ideas for each month-subject combo
                prompt_template=idea_chat_prompt_template, 
                parser=activity_ideas_parser,
                model=activity_ideas_model,
                num_ideas=num_plans,
                month=month,
                theme=theme,
                subject=subject, 
                topic=topic
            )
            print("  - Generated ideas")
            res[month][subject] = activity_ideas_parsed.dict()
            
            for idea in res[month][subject]["ideas"]: #  Generate lesson plans for each idea
                activity_name = idea["name"]
                activity_description = idea["description"]
                lesson_plan = prompt_wrapper(
                    prompt_template=lesson_plan_chat_prompt_template, 
                    parser=lesson_plan_parser,
                    model=lesson_plan_model,
                    activity_name=activity_name, 
                    activity_description=activity_description, 
                    subject=subject
                )
                idea.update(lesson_plan)
                lesson_plan_markdown = convert_lesson_plan_to_markdown(idea)
                markdown_lst.append(f"\n{lesson_plan_markdown}")
                markdown_string = "\n".join(markdown_lst)
            print("  - Generated lesson plans")
                
        # Write the output of each month to a Markdown file
        output_file_path = f'../output/{idx + 1} {month} Lesson Plans.md'
        with open(output_file_path, "w") as output_file:
            output_file.write(markdown_string)

        print(f"Markdown output written to '{output_file_path}'.")
        print("*" * 50, "\n")
        # break
    
    # Write full results to a JSON file
    full_result_output_path = "../output/lesson_plan.json"
    with open(full_result_output_path , "w") as json_file:
        json.dump(res, json_file)
    print(f"JSON output written to '{full_result_output_path }'.")
    return res

# Run

In [49]:
annual_plan_df.head(6)

,month,theme_to_explore,language_arts,math,social_studies,science
0,August,Getting to Know One Another; Class Community,"Personal narratives, reading comprehension",Counting and sorting,Our classroom community; citizenship and gover...,Exploring the school environment and local eco...
1,September,"On the Map, where we’re from",Descriptive writing (My Place); vocabulary dev...,latitutde and longitude; place value,My hometown; local history; geography,"Water cycle, rivers; Geology, caves, aquifers"
2,October,Fall; Halloween; Dia de los Muertos,Creative writing; grammar and punctuation,"units of measurement (length, weight, etc.); E...",Fall celebrations around the world; elections,Food chains and food webs; Local flora and fauna
3,November,Gratitude; Native Americans,Listening and speaking skills; Thankful Though...,Addition and subtraction,Native American history; Settlement of US and ...,Seasonal Changes and Plants; Winter Animal Ada...
4,December,Winter Solstice and winter holidays,book exchange; holiday cards; holiday play/skit,Geometry (2-D shapes); symmetry (snowflake),Winter holidays worldwide,"Physical science (matter, energy, force, motion)"


In [50]:
activity_ideas_model = ChatOpenAI(model_name=model_name, temperature=.8, max_tokens=500)

In [51]:
lesson_plan_model = ChatOpenAI(model_name=model_name, temperature=.2, max_tokens=3000)

In [ ]:
lesson_plans = wrapper(annual_plan_df, activity_ideas_model, lesson_plan_model, months=["December"])

,month,theme_to_explore,language_arts,math,social_studies,science
4,December,Winter Solstice and winter holidays,book exchange; holiday cards; holiday play/skit,Geometry (2-D shapes); symmetry (snowflake),Winter holidays worldwide,"Physical science (matter, energy, force, motion)"


# December
## Language Arts
  - Generated ideas


# View Results

In [ ]:
month = months[0]

In [ ]:
for idea in lesson_plans[month]['language_arts']['ideas']:
    display(Markdown(f"**{idea['name']}**: {idea['description']}"))

In [ ]:
display(JSON(lesson_plans))